# Price Spike Window Size Testing

This notebook tests different window sizes for the price spike calculation in a mean reversion strategy.

In [ ]:
import sys
import os

# Import our backtester package
sys.path.append(os.path.abspath('../../'))
from backtester import get_price_data, get_vwap
print("Using backtester package")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Try to import seaborn, but don't fail if it's not available
try:
    import seaborn as sns
    print(f"Seaborn version: {sns.__version__}")
    sns.set(style="whitegrid")
    HAS_SEABORN = True
except ImportError:
    print("Seaborn not available, using matplotlib instead")
    HAS_SEABORN = False

## 1. Load Data

First, let's load the Squid_Ink price data and limit it to the first 20,000 timestamps (in-sample data).

In [ ]:
# Load data directly using backtester package
print("Loading price data...")
prices = get_price_data('SQUID_INK', 1)
print(f"Loaded {len(prices)} price data points")

# Limit to first 20,000 timestamps (in-sample data)
in_sample_prices = prices.iloc[:20000]
print(f"Limited to {len(in_sample_prices)} in-sample data points")

# Get VWAP
print("Getting VWAP for SQUID_INK...")
squid_vwap = in_sample_prices['vwap']
print(f"Got VWAP with {len(squid_vwap)} data points")
print(f"VWAP range: {squid_vwap.min()} to {squid_vwap.max()}")

# Calculate log prices and log returns
log_prices = np.log(squid_vwap)
log_returns = log_prices.diff().dropna()
print(f"Calculated log returns with {len(log_returns)} data points")

# Calculate regular returns for strategy evaluation
returns = squid_vwap.pct_change().dropna()
print(f"Calculated returns with {len(returns)} data points")

## 2. Calculate Price Spikes

Let's calculate price spikes as log returns divided by rolling standard deviation.

In [ ]:
def calculate_price_spikes(log_returns, window=20):
    """
    Calculate price spikes as log returns divided by rolling standard deviation.
    
    Parameters:
        log_returns (pd.Series): Series of log returns
        window (int): Window size for rolling standard deviation
        
    Returns:
        pd.Series: Price spikes (z-scores)
    """
    # Calculate rolling standard deviation
    rolling_std = log_returns.rolling(window=window).std()
    
    # Calculate price spikes (z-scores)
    price_spikes = log_returns / rolling_std
    
    return price_spikes

# Calculate price spikes with different window sizes
window_sizes = [5, 10, 20, 50, 100]
price_spike_indicators = {}

for window in window_sizes:
    price_spike_indicators[f'Spike_{window}'] = calculate_price_spikes(log_returns, window)

## 3. Define Price Spike-Based Mean Reversion Strategy

Let's define a strategy that only buys below 2000 when there's a downward price spike and only sells above 2000 when there's an upward price spike.

In [ ]:
def price_spike_strategy(prices, spikes, fair_price, upper_threshold=2.0, lower_threshold=-2.0, holding_period=10):
    """
    Implement a price spike-based mean reversion strategy.
    
    Parameters:
        prices (pd.Series): Series of prices
        spikes (pd.Series): Series of price spikes (z-scores)
        fair_price (float): Fair price to revert to
        upper_threshold (float): Upper threshold for price spikes
        lower_threshold (float): Lower threshold for price spikes
        holding_period (int): Number of periods to hold the position
        
    Returns:
        pd.Series: Portfolio positions (1 for long, -1 for short, 0 for no position)
    """
    # Initialize positions
    positions = pd.Series(0, index=prices.index)
    
    # Get valid indices where spikes is not NaN
    valid_indices = spikes.dropna().index
    
    # Set positions based on price spikes and fair price
    for time in valid_indices:
        # Get the current price and spike value
        current_price = prices.loc[time]
        current_spike = spikes.loc[time]
        
        # Get the index position
        idx = prices.index.get_loc(time)
        
        # ONLY short above fair price when there's an upward price spike
        if current_price > fair_price and current_spike > upper_threshold:
            # Set short position for holding period
            end_idx = min(idx + holding_period + 1, len(positions))
            positions.iloc[idx+1:end_idx] = -1
        
        # ONLY buy below fair price when there's a downward price spike
        elif current_price < fair_price and current_spike < lower_threshold:
            # Set long position for holding period
            end_idx = min(idx + holding_period + 1, len(positions))
            positions.iloc[idx+1:end_idx] = 1
    
    return positions

## 4. Test Different Window Sizes

Let's test different window sizes for the price spike calculation, using fixed threshold parameters.

In [ ]:
# Define the fair price
FAIR_PRICE = 2000

# Define fixed threshold parameters
upper_threshold = 2.0
lower_threshold = -2.0
holding_period = 10

# Initialize results dictionary
window_results = {}

# Test different window sizes
for window in window_sizes:
    spikes = price_spike_indicators[f'Spike_{window}']
    
    # Get positions
    positions = price_spike_strategy(squid_vwap, spikes, FAIR_PRICE, upper_threshold, lower_threshold, holding_period)
    
    # Calculate strategy returns
    strategy_returns = positions.shift(1) * returns
    strategy_returns = strategy_returns.dropna()
    
    # Calculate performance metrics
    total_return = strategy_returns.sum()
    sharpe_ratio = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252)  # Annualized
    win_rate = (strategy_returns > 0).mean()
    
    # Count the number of trades
    num_trades = (positions.diff() != 0).sum()
    
    # Store results
    window_results[window] = {
        'Window Size': window,
        'Total Return': total_return,
        'Sharpe Ratio': sharpe_ratio,
        'Win Rate': win_rate,
        'Number of Trades': num_trades
    }

# Convert results to DataFrame
window_df = pd.DataFrame(window_results).T

# Sort by Total Return
window_df = window_df.sort_values('Total Return', ascending=False)

# Display results
window_df

## 5. Visualize Window Size Results

Let's visualize the results of the window size testing.

In [ ]:
# Plot performance metrics by window size
plt.figure(figsize=(15, 10))

plt.subplot(2, 2, 1)
plt.plot(window_df.index, window_df['Total Return'], marker='o')
plt.title('Total Return by Window Size')
plt.xlabel('Window Size')
plt.ylabel('Total Return')
plt.grid(True)

plt.subplot(2, 2, 2)
plt.plot(window_df.index, window_df['Sharpe Ratio'], marker='o')
plt.title('Sharpe Ratio by Window Size')
plt.xlabel('Window Size')
plt.ylabel('Sharpe Ratio')
plt.grid(True)

plt.subplot(2, 2, 3)
plt.plot(window_df.index, window_df['Win Rate'], marker='o')
plt.title('Win Rate by Window Size')
plt.xlabel('Window Size')
plt.ylabel('Win Rate')
plt.grid(True)

plt.subplot(2, 2, 4)
plt.plot(window_df.index, window_df['Number of Trades'], marker='o')
plt.title('Number of Trades by Window Size')
plt.xlabel('Window Size')
plt.ylabel('Number of Trades')
plt.grid(True)

plt.tight_layout()
plt.show()

## 6. Analyze Best Window Size

Let's analyze the performance of the strategy with the best window size based on total returns.

In [ ]:
# Get the best window size based on total returns
best_window = window_df.index[0]

print(f'Best window size based on total returns: {best_window}')
print(f'Total Return: {window_df.loc[best_window, "Total Return"]:.4f}')
print(f'Sharpe Ratio: {window_df.loc[best_window, "Sharpe Ratio"]:.4f}')
print(f'Win Rate: {window_df.loc[best_window, "Win Rate"]:.4f}')
print(f'Number of Trades: {int(window_df.loc[best_window, "Number of Trades"])}')

# Get positions for the best window size
best_spikes = price_spike_indicators[f'Spike_{best_window}']
best_positions = price_spike_strategy(squid_vwap, best_spikes, FAIR_PRICE, upper_threshold, lower_threshold, holding_period)

# Calculate strategy returns
best_returns = best_positions.shift(1) * returns
best_returns = best_returns.dropna()

# Plot cumulative returns
plt.figure(figsize=(15, 10))

plt.subplot(2, 1, 1)
plt.plot(best_returns.cumsum(), label=f'Strategy Returns (Window={best_window})')
plt.title(f'Cumulative Returns of Best Price Spike-Based Strategy', fontsize=16)
plt.legend(fontsize=12)
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(squid_vwap, label='VWAP')
plt.axhline(y=FAIR_PRICE, color='r', linestyle='--', label='Fair Price (2000)')

# Plot buy and sell signals
buy_signals = best_positions.diff() > 0
sell_signals = best_positions.diff() < 0

plt.scatter(buy_signals[buy_signals].index, squid_vwap[buy_signals], 
            marker='^', s=100, color='green', label='Buy Signal (Below 2000)')
plt.scatter(sell_signals[sell_signals].index, squid_vwap[sell_signals], 
            marker='v', s=100, color='red', label='Sell Signal (Above 2000)')

plt.title('Price with Trading Signals', fontsize=16)
plt.legend(fontsize=12)
plt.grid(True)

plt.tight_layout()
plt.show()

## 7. Conclusion

In this notebook, we've tested different window sizes for the price spike calculation in a mean reversion strategy. The strategy only buys below 2000 when there's a downward price spike and only sells above 2000 when there's an upward price spike.

Key findings:
1. The window size for calculating price spikes has a significant impact on the strategy's performance
2. The optimal window size based on total returns is [to be filled after running]
3. Smaller window sizes are more responsive to recent price movements but may generate more false signals
4. Larger window sizes are more stable but may miss short-term opportunities
5. The best window size balances the trade-off between responsiveness and stability

These findings suggest that the window size is a critical parameter for price spike-based mean reversion strategies. By optimizing the window size, we can improve the quality of our price spike signals and potentially increase the profitability of the strategy.